In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Read the Excel-format file (actually a tab-delimited .txt table)
df = pd.read_table('Area_0_20256141831.txt')

# Adjust header rows
df = df.iloc[3:].reset_index(drop=True)  # Skip first 3 lines (usually metadata)
new_columns = df.iloc[0]  # Use the first row as column names
df.columns = new_columns
df = df.iloc[1:].reset_index(drop=True)  # Remove the row used as header

# Handle duplicated column names
df.columns = pd.Series(df.columns).astype(str)
df.columns = df.columns.where(~df.columns.duplicated(), df.columns + '_' + df.columns.duplicated(keep='first').cumsum().astype(str))

# Filter rows that have non-empty MS/MS spectrum information
df1 = df[df['MS/MS spectrum'].notna()].copy()

# Convert 'S/N average' column to numeric values
df1['S/N average'] = pd.to_numeric(df1['S/N average'], errors='coerce')

# Remove rows where signal-to-noise ratio < 3
df1 = df1[df1['S/N average'] >= 3].copy()

# Define replicate sample columns to convert to numeric
cols_to_convert = ['0-1', '0-2', '0-3', '1-1', '1-2', '1-3',
                   '2-1', '2-2', '2-3', '3-1', '3-2', '3-3',
                   '4-1', '4-2', '4-3']

# Convert intensity columns to numeric
df1[cols_to_convert] = df1[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Further filter: keep rows where '4-1' intensity is greater than or equal to 1000
df1 = df1[df1['4-1'] >= 1e3].copy()

# Define column names for the control (0 group) and treatment (4 group)
low_group = ['0-1', '0-2', '0-3']
high_group = ['4-1', '4-2', '4-3']

# Ensure all group columns are numeric
df1[low_group + high_group] = df1[low_group + high_group].apply(pd.to_numeric, errors='coerce')

# Initialize lists to store p-values and log2 fold changes
p_values = []
log2fc_values = []

# Iterate through each row to perform t-test and calculate log2FC
for idx, row in df1.iterrows():
    try:
        group_low = row[low_group].astype(float).values
        group_high = row[high_group].astype(float).values

        # Skip rows with missing values
        if np.isnan(group_low).any() or np.isnan(group_high).any():
            raise ValueError

        # Perform two-sided t-test
        pval = ttest_ind(group_high, group_low, equal_var=False).pvalue

        # Calculate log2 fold change, with small constant to avoid division by zero
        mean_low = np.mean(group_low)
        mean_high = np.mean(group_high)
        log2fc = np.log2((mean_high + 1e-9) / (mean_low + 1e-9))

    except:
        pval = np.nan
        log2fc = np.nan

    p_values.append(pval)
    log2fc_values.append(log2fc)

# Add results to the dataframe
df1['p_value'] = p_values
df1['log2FC'] = log2fc_values

############## Curation steps ##############

# Remove isotopic peaks; retain only parent ions
df = df1
df['Post curation result'] = df['Post curation result'].astype(str)
df = df[~df['Post curation result'].str.contains("found in higher mz's MsMs")].reset_index(drop=True)

# Remove redundant features (within mz ±0.005 and RT ±0.2)
df['Average Mz'] = pd.to_numeric(df['Average Mz'], errors='coerce')
df['Average Rt(min)'] = pd.to_numeric(df['Average Rt(min)'], errors='coerce')
df['4-1'] = df['4-1'].astype(float)
df = df.sort_values('Average Mz')

# Initialize storage for filtered features
groups = []
while not df.empty:
    mz0 = df.iloc[0]['Average Mz']
    rt0 = df.iloc[0]['Average Rt(min)']
    
    # Identify nearby features based on mz and RT thresholds
    mask = (df['Average Mz'].sub(mz0).abs() <= 0.005) & (df['Average Rt(min)'].sub(rt0).abs() <= 0.2)
    group = df[mask]
    
    if not group.empty:
        # Retain the feature with the highest intensity in group '4-1'
        best_row = group.loc[group['4-1'].idxmax()]
        groups.append(best_row)

    # Remove the processed group from dataframe
    df = df[~mask]

# Compile and export the curated result
result_df = pd.DataFrame(groups)
output = "result.xlsx"
result_df.to_excel(output, index=False)
